In [6]:
!python run_recbole_gnn.py > gnn.log

Traceback (most recent call last):
  File "run_recbole_gnn.py", line 3, in <module>
    from recbole_gnn.quick_start import run_recbole_gnn
  File "d:\code\BITSheJianDianPing\AI\Recommendation\recbole_gnn\quick_start.py", line 3, in <module>
    from recbole.utils import init_logger, init_seed, set_color
  File "d:\code\BITSheJianDianPing\AI\Recommendation\recbole\utils\__init__.py", line 1, in <module>
    from recbole.utils.logger import init_logger, set_color
  File "d:\code\BITSheJianDianPing\AI\Recommendation\recbole\utils\logger.py", line 26, in <module>
    from recbole.utils.utils import get_local_time, ensure_dir
  File "d:\code\BITSheJianDianPing\AI\Recommendation\recbole\utils\utils.py", line 22, in <module>
    import torch
  File "c:\Users\srx\.conda\envs\recbole\lib\site-packages\torch\__init__.py", line 129, in <module>
    raise err
OSError: [WinError 1455] 页面文件太小，无法完成操作。 Error loading "c:\Users\srx\.conda\envs\recbole\lib\site-packages\torch\lib\shm.dll" or one of its 

In [7]:
from logging import getLogger
from recbole_gnn.config import Config
from recbole_gnn.utils import create_dataset, data_preparation, get_model, get_trainer
from recbole.utils import (
    init_logger,
    init_seed,
)

In [8]:
def load_data_and_model(model_file):
    r"""Load filtered dataset, split dataloaders and saved model.

    Args:
        model_file (str): The path of saved model file.

    Returns:
        tuple:
            - config (Config): An instance object of Config, which record parameter information in :attr:`model_file`.
            - model (AbstractRecommender): The model load from :attr:`model_file`.
            - dataset (Dataset): The filtered dataset.
            - train_data (AbstractDataLoader): The dataloader for training.
            - valid_data (AbstractDataLoader): The dataloader for validation.
            - test_data (AbstractDataLoader): The dataloader for testing.
    """
    import torch

    checkpoint = torch.load(model_file)
    config = checkpoint["config"]
    init_seed(config["seed"], config["reproducibility"])
    init_logger(config)
    logger = getLogger()
    logger.info(config)

    dataset = create_dataset(config)
    logger.info(dataset)
    train_data, valid_data, test_data = data_preparation(config, dataset)

    init_seed(config["seed"], config["reproducibility"])
    model = get_model(config["model"])(config, train_data._dataset).to(config["device"])
    model.load_state_dict(checkpoint["state_dict"])
    model.load_other_parameter(checkpoint.get("other_parameter"))

    return config, model, dataset, train_data, valid_data, test_data

In [9]:
from recbole.utils.case_study import full_sort_scores, full_sort_topk
# from recbole.quick_start import load_data_and_model
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file='saved/' + input(),
    # model_file='saved/SimGCL-May-21-2023_22-12-29.pth',
)  # Here you can replace it by your model path.
uid_series = dataset.token2id(dataset.uid_field, [str(user_id) for user_id in range(1, dataset.user_num)])
top30 = full_sort_topk(uid_series, model, test_data, k=30, device=config['device'])

11 Jun 11:41    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = d:\RecBole\recbole\config\../dataset_example/ml-100k
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Par

len of value 52586
len of value 52586


11 Jun 11:41    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
11 Jun 11:41    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}]


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

In [ ]:
import pandas as pd
dish_info = pd.read_csv("dataset/dish.csv")
user_info = pd.read_csv("dataset/user.csv")

In [ ]:
recommendation_pd = pd.DataFrame(columns=["user_id", "dish_id", "rec_score","dish_name","dish_flavor"])

for user_id in range(1, dataset.user_num):
    user_flavor = user_info[user_info["user_id:token"] == user_id]["dislike:token"].values[0]
    indexes = (top30.indices[user_id - 1] + 1).cpu()
    scores = top30.values[user_id - 1].cpu()
    scores = (scores - scores[scores != float('-inf')].min()) / (max(scores) - scores[scores != float('-inf')].min())
    for index, score in zip(indexes, scores):
        if(score == float('-inf') or len(dish_info[dish_info["item_id:token"] == int(index)]["flavor:token"].values) == 0 or dish_info[dish_info["item_id:token"] == int(index)]["flavor:token"].values[0] == user_flavor): continue
        recommendation_pd = recommendation_pd.append({"user_id": user_id, "dish_id": int(index), "rec_score": float(score), "dish_name": dish_info[dish_info["item_id:token"] == int(index)]["name:token"].values[0], "dish_flavor": dish_info[dish_info["item_id:token"] == int(index)]["flavor:token"].values[0]}, ignore_index=True)

In [ ]:
user_info[user_info["user_id:token"] == user_id]

,user_id:token,name:token,flavor:token,dislike:token
99,100,吴莹,鲜,辣


In [ ]:
recommendation_pd

,user_id,dish_id,rec_score,dish_name,dish_flavor
0,1,23,0.753092,烤肉拌饭,咸
1,1,26,0.693098,红烧牛肉面,咸
2,1,10,0.594351,酸菜鱼,酸
3,1,13,0.553330,炸酱面,咸
4,1,29,0.514412,酸辣土豆丝,酸
...,...,...,...,...,...
1841,100,17,0.397185,拔丝地瓜,甜
1842,100,5,0.392224,蒜蓉西兰花,鲜
1843,100,20,0.313862,芝士焗土豆,甜
1844,100,8,0.156359,糖醋里脊,甜


In [ ]:
# user_id = 1
# index = (top30.indices[user_id - 1] + 1).cpu()
# score = top30.values[user_id - 1].cpu()
# # max/min归一化sccore，忽略-inf
# score = (score - score[score != float('-inf')].min()) / (max(score) - score[score != float('-inf')].min())
# # 将-inf替换为0
# score[score == float('-inf')] = 0
# recommendation_pd = recommendation_pd.append(
#     pd.DataFrame({
#         "user_id": [user_id] * len(index),
#         "dish_id": index,
#         "rec_score": score,
#     })
# )

In [ ]:
# recommendation_pd = pd.DataFrame(columns=["user_id", "dish_id", "rec_score"])

In [ ]:
# tmp = dish_info[((dish_info["item_id:token"].isin(index)) & (dish_info["flavor:token"] != user_info[user_info["user_id:token"] == user_id]["dislike:token"].values[0]))][:10][["item_id:token"]]
# # item_id:token列改名为dish_id
# tmp.rename(columns={"item_id:token": "dish_id"}, inplace=True)
# # # tmp添加一列"user_id"为1
# tmp["user_id"] = user_id
# recommendation_pd.append(tmp)

In [ ]:
user_id = 20
recommendation_pd[recommendation_pd["user_id"] == user_id].sort_values(by="rec_score", ascending=False)

,user_id,dish_id,rec_score,dish_name,dish_flavor
338,20,17,1.000000,拔丝地瓜,甜
339,20,30,0.388607,苦瓜炒蛋,苦
340,20,16,0.321158,炒年糕,甜
341,20,25,0.305862,奶油蘑菇汤,甜
342,20,3,0.290109,水煮肉片,辣
343,20,6,0.289570,番茄炒蛋,甜
344,20,9,0.273413,酸辣汤,酸
345,20,22,0.255855,酸奶冻,甜
346,20,18,0.249815,鱼香茄子,酸
347,20,4,0.236066,麻婆豆腐,辣


In [ ]:
recommendation_pd[["user_id", "dish_id", "rec_score"]].to_csv("recommendation.csv", index=False)

In [ ]:
recommendation_pd

,user_id,dish_id,rec_score,dish_name,dish_flavor
0,1,23,0.753092,烤肉拌饭,咸
1,1,26,0.693098,红烧牛肉面,咸
2,1,10,0.594351,酸菜鱼,酸
3,1,13,0.553330,炸酱面,咸
4,1,29,0.514412,酸辣土豆丝,酸
...,...,...,...,...,...
1841,100,17,0.397185,拔丝地瓜,甜
1842,100,5,0.392224,蒜蓉西兰花,鲜
1843,100,20,0.313862,芝士焗土豆,甜
1844,100,8,0.156359,糖醋里脊,甜


In [ ]:
inter_info = pd.read_csv("recbole/dataset_example/ml-100k/ml-100k.inter", sep="\t")
inter_info[inter_info["user_id:token"] == user_id]["item_id:token"].map(dish_info.set_index("item_id:token")["name:token"]+"("+dish_info.set_index("item_id:token")["flavor:token"]+")").value_counts()

芝士焗土豆(甜)    253
糖醋里脊(甜)      36
炒年糕(甜)       34
酸奶冻(甜)       30
提拉米苏(甜)      27
芝士蛋糕(甜)      25
番茄炒蛋(甜)      22
拔丝地瓜(甜)      16
酸辣土豆丝(酸)     16
鱼香茄子(酸)      16
奶油蘑菇汤(甜)     11
蒜蓉西兰花(鲜)     10
红烧排骨(甜)       9
苦瓜炒蛋(苦)       8
麻婆豆腐(辣)       7
糖醋排骨(甜)       7
香辣蟹(辣)        6
酸菜鱼(酸)        6
鱼香肉丝(酸)       5
水煮肉片(辣)       4
麻辣香锅(辣)       4
芒果布丁(甜)       4
干锅牛蛙(辣)       2
酸辣汤(酸)        2
Name: item_id:token, dtype: int64

In [ ]:
user_info[user_info["user_id:token"] == user_id][["flavor:token", "dislike:token"]]

,flavor:token,dislike:token
19,甜,咸


In [ ]:
# dish_info[dish_info["item_id:token"].isin(index)][["name:token", "flavor:token"]]
# dish_info[((dish_info["item_id:token"].isin(index)) & (dish_info["flavor:token"] != user_info[user_info["user_id:token"] == user_id]["dislike:token"].values[0]))][["item_id:token", "name:token", "flavor:token"]]
# dish_info[dish_info["item_id:token"].isin(index)][["name:token", "flavor:token"]]